#### practice

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('mytree').getOrCreate()

In [3]:
from pyspark.ml.pipeline import Pipeline

In [4]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier

In [19]:
data = spark.read.csv('College.csv', inferSchema = True, header = True)

In [20]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [22]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [21]:
from pyspark.ml.feature import VectorAssembler

In [23]:
assembler = VectorAssembler(inputCols = ['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'], outputCol = 'features')

In [24]:
output = assembler.transform(data)

In [25]:
from pyspark.ml.feature import StringIndexer

In [26]:
indexer = StringIndexer(inputCol='Private', outputCol = 'private_idx')

In [27]:
output_fixed = indexer.fit(output).transform(output)

In [28]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- private_idx: double (nullable = false)



In [29]:
final_data = output_fixed.select('features','private_idx')

In [30]:
train, test = final_data.randomSplit([0.7, 0.3])

In [31]:
rf = RandomForestClassifier(labelCol='private_idx')
gbt = GBTClassifier(labelCol='private_idx')
dt = DecisionTreeClassifier(labelCol='private_idx')

In [32]:
dtc = dt.fit(train)
rfc = rf.fit(train)
gbtc = gbt.fit(train)

In [33]:
dt_pred = dtc.transform(test)
rf_pred = rfc.transform(test)
gbt_pred = gbtc.transform(test)

In [34]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [35]:
binary_eval = BinaryClassificationEvaluator(labelCol='private_idx')

In [36]:
binary_eval.evaluate(dt_pred) #auc, default

0.911570158317068

In [37]:
binary_eval.evaluate(rf_pred)

0.959119496855346

In [39]:
binary_eval2 = BinaryClassificationEvaluator(labelCol='private_idx', rawPredictionCol='prediction')

In [40]:
binary_eval2.evaluate(gbt_pred)

0.8455324224680113

In [41]:
acc_eval = MulticlassClassificationEvaluator(labelCol='private_idx', metricName='accuracy')

In [42]:
acc_eval.evaluate(dt_pred)

0.9078341013824884

In [43]:
acc_eval.evaluate(rf_pred)

0.9032258064516129

In [44]:
acc_eval.evaluate(gbt_pred)

0.8940092165898618

#### consulting project

In [45]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('treeproject').getOrCreate()

In [46]:
data = spark.read.csv('dog_food.csv', inferSchema = True, header = True)

In [47]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [48]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [49]:
assembler = VectorAssembler(inputCols = ['A', 'B', 'C', 'D'], outputCol = 'features')
output = assembler.transform(data)

In [50]:
rf = RandomForestClassifier(labelCol='Spoiled', featuresCol='features')
final_data = output.select('features','Spoiled')
rfc = rf.fit(final_data)

In [51]:
rfc.featureImportances

SparseVector(4, {0: 0.0256, 1: 0.0204, 2: 0.93, 3: 0.024})